## Tutela Market Share Analysis

Datasets Required:
- Tutela
- ISED Grid

In [1]:
#-----------------
#Import Statements
#-----------------

import os
import awswrangler as wr
import boto3
import pandas as pd
import numpy as np
import ray

In [2]:
#-----------------
#AWS directories
#-----------------
s3_root = 's3://crc-data-lake-research/sma/ryan_fabek'
s3_structured_root = 's3://crc-data-lake-structured/tutela'

structured_database = 'crc_data_lake_structured'
research_database = 'crc_data_lake_research'

results_path = '{}/tutela_market_share'.format(s3_root)

In [3]:
session = boto3.Session(profile_name='ssm-dev')
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': 'tcp://127.0.0.1:65142',
 'raylet_socket_name': 'tcp://127.0.0.1:61011',
 'webui_url': None,
 'session_dir': 'C:\\Users\\RYAN~1.FAB\\AppData\\Local\\Temp\\2\\ray\\session_2022-03-29_10-06-23_145476_8128',
 'metrics_export_port': 62923,
 'gcs_address': '127.0.0.1:61552',
 'address': '127.0.0.1:61552',
 'node_id': '29041080322dfded26c5dbf7421e51887921e70e4f50346c6a2d1787'}

In [6]:
tutela_market_share_query = """

WITH tutela_clean AS(
SELECT device_simserviceproviderbrandname,location_latitude,location_longitude,country,connection_technology
FROM
    crc_data_lake_structured.tutela
WHERE
    country = 'Canada' AND
    connection_technology IN ('LTE', 'LTE_CA')
    AND tutela.connection_mobilefrequency IS NOT NULL
    AND tutela.connection_serviceproviderbrandname IS NOT NULL
    AND device_simserviceproviderbrandname = :provider;
),

grid AS (
SELECT tier_2,tier_3,tier_4,geometry
FROM
    crc_data_lake_structured.ised_grid
WHERE
    tier_2 = :tier_2;

),

prov_and_terr AS(

SELECT area_label,area_name
FROM
    crc_data_lake_structured.spectrum_service_area

)

SELECT *
FROM
    tutela_clean
JOIN
      grid
      ON ST_Within(
        ST_Point(tutela_clean.location_longitude, tutela_clean.location_latitude),
        ST_GeometryFromText(grid.geometry)
      )
LEFT JOIN
    prov_and_terr
ON
    grid.tier_2 = prov_and_terr.area_label

"""

parameters_query_tier_2 = """

SELECT DISTINCT tier_2 as tier
FROM
    crc_data_lake_structured.ised_grid
ORDER BY
    tier_2

"""
parameters_query_provider = """

SELECT DISTINCT device_simserviceproviderbrandname as provider
FROM
    crc_data_lake_structured.tutela
WHERE
    tutela.country = 'Canada' AND
    tutela.connection_technology IN ('LTE', 'LTE_CA')
    AND tutela.connection_mobilefrequency IS NOT NULL
    AND tutela.connection_serviceproviderbrandname IS NOT NULL
ORDER BY
    device_simserviceproviderbrandname

"""

tier_2s = wr.athena.read_sql_query(
                                parameters_query_tier_2,
                                database=research_database,
                                s3_output=s3_root,
                                keep_files=False,
                                boto3_session=session,
                                ctas_approach=False)


providers = wr.athena.read_sql_query(
                                parameters_query_provider,
                                database=research_database,
                                s3_output=s3_root,
                                keep_files=False,
                                boto3_session=session,
                                ctas_approach=False)
                            

providers

for i in range(0,len(tier_2s.index)):

    tier_2 = tier_2s.iloc[i]['tier']

    for j in range(0,len(providers.index)):

        provider = providers.iloc[j]['provider']

        


        parameters = {'tier_2' : '\'' + tier_2 + '\'','provider' : '\'' + provider + '\'' }

        results = wr.athena.read_sql_query(
                                tutela_market_share_query,
                                database=research_database,
                                s3_output=s3_root,
                                keep_files=False,
                                boto3_session=session,
                                ctas_approach=False,
                                params=parameters
                            )
        if len(results.index) != 0:

            wr.s3.to_parquet(
            df = results,
            path = results_path + '/' + tier_2 + '/' + provider,
            dataset = True,
            mode = 'overwrite'
        )
        

KeyboardInterrupt: 

In [7]:
wr.s3.read_parquet(

path = 's3://crc-data-lake-research/sma/ryan_fabek/tutela_market_share/2-001/3/056c3d366f664131a0ecebc5f1d7579d.snappy.parquet'

)

device_simserviceproviderbrandname  location_latitude  location_longitude  \
0                                   3          47.793079          -53.976532   
1                                   3          47.165042          -55.135903   
2                                   3          47.164978          -55.136805   
3                                   3          47.165042          -55.135903   
4                                   3          47.165042          -55.135903   
5                                   3          47.164978          -55.136805   
6                                   3          47.163583          -55.142638   
7                                   3          47.164978          -55.136805   
8                                   3          47.190542          -55.131906   
9                                   3          47.165042          -55.135903   
10                                  3          47.165042          -55.135903   
11                                  3          47.163417          -55.152349   
12                                  3          47.165042          -55.135903   
13                                  3          47.165042          -55.135903   
14                                  3          47.793079          -53.976532   
15                                  3          47.164755          -55.135008   
16                                  3          47.165042          -55.135903   
17                                  3          47.165042          -55.135903   
18                                  3          47.163627          -55.141627   
19                                  3          47.163417          -55.152349   
20                                  3          47.165042          -55.135903   
21                                  3          47.165042          -55.135903   
22                                  3          47.165042          -55.135903   
23                                  3          47.164978          -55.136805   
24                                  3          47.190542          -55.131906   
25                                  3          47.582916          -52.734515   

   country connection_technology tier_2 tier_3 tier_4  \
0   Canada                   LTE  2-001  3-001  4-002   
1   Canada                   LTE  2-001  3-001  4-003   
2   Canada                   LTE  2-001  3-001  4-003   
3   Canada                   LTE  2-001  3-001  4-003   
4   Canada                   LTE  2-001  3-001  4-003   
5   Canada                   LTE  2-001  3-001  4-003   
6   Canada                   LTE  2-001  3-001  4-003   
7   Canada                   LTE  2-001  3-001  4-003   
8   Canada                   LTE  2-001  3-001  4-003   
9   Canada                   LTE  2-001  3-001  4-003   
10  Canada                   LTE  2-001  3-001  4-003   
11  Canada                   LTE  2-001  3-001  4-003   
12  Canada                   LTE  2-001  3-001  4-003   
13  Canada                   LTE  2-001  3-001  4-003   
14  Canada                   LTE  2-001  3-001  4-002   
15  Canada                   LTE  2-001  3-001  4-003   
16  Canada                   LTE  2-001  3-001  4-003   
17  Canada                   LTE  2-001  3-001  4-003   
18  Canada                   LTE  2-001  3-001  4-003   
19  Canada                   LTE  2-001  3-001  4-003   
20  Canada                   LTE  2-001  3-001  4-003   
21  Canada                   LTE  2-001  3-001  4-003   
22  Canada                   LTE  2-001  3-001  4-003   
23  Canada                   LTE  2-001  3-001  4-003   
24  Canada                   LTE  2-001  3-001  4-003   
25  Canada                   LTE  2-001  3-001  4-001   

                                             geometry area_label  \
0   POLYGON ((-53.98333300000002 47.80000000000280...      2-001   
1   POLYGON ((-55.150000000000084 47.1666670000028...      2-001   
2   POLYGON ((-55.150000000000084 47.1666670000028...      2-001   
3   POLYGON ((-55.150000000000